# Working with AGOL items - *Streamlined*

---
Fall 2024  
John.Fay@duke.edu

---

## Importing packages
The `arcgis` package is quite large, so we import just the items we need from the appropriate submodules:

| Submodule | Package | Uses |
| :--- | :--- | :--- |
| **Features** | FeatureLayer | Create a FeatureLayer object directly from its URL |
|  | GeoAccessor | Create a Spatial Dataframe from a layer or local file |
| **geocoding** | geocode | geocode: convert location names to geometries |
| **geometry** | Point | the Point class |
|   | filters | spatial filters for spatial queries |


In [ ]:
#Import basic packages
import pandas as pd

#Import the arcgis "GIS" object and connect to AGOL
from arcgis import GIS
gis = GIS()

#Import other classes
from arcgis.features import FeatureLayer, GeoAccessor
from arcgis.raster import Raster, ImageryLayer
from arcgis.mapping import MapServiceLayer, MapRasterLayer, MapImageLayer
from arcgis.geocoding import geocode
from arcgis.geometry import Point, filters

## Finding Data
* Flood Hazard$^*$ - https://dukeuniv.maps.arcgis.com/home/item.html?id=11955f1b47ec41a3af86650824e0c634
* Sea Level Rise$^*$ - https://dukeuniv.maps.arcgis.com/home/item.html?id=1311369dea53479ca9608cb5abca565f
* Storm Surge$^*$$^*$ - https://coast.noaa.gov/arcgis/rest/services/FloodExposureMapper/CFEM_CoastalFloodHazardComposite/MapServer/20

$^*$ *Search the Living Atlas*  
$^*$$^*$ *Google Search:* `Coastal Flood Hazard Composite arcgis/rest` -> [Link](https://coast.noaa.gov/arcgis/rest/services/FloodExposureMapper/CFEM_CoastalFloodHazardComposite/MapServer)

In [ ]:
flood_item = gis.content.get('11955f1b47ec41a3af86650824e0c634')

slr_item = gis.content.get('1311369dea53479ca9608cb5abca565f')

In [ ]:
#Get the surge item as a map raster layer
the_url='https://coast.noaa.gov/arcgis/rest/services/FloodExposureMapper/CFEM_CoastalFloodHazardComposite/MapServer/20'
ms_layer = MapImageLayer(the_url)
type(ms_layer)

In [ ]:
ms_layer.manager

In [ ]:
#Get its layer
flood_layer = flood_item.layers[0]
flood_layer.url

In [ ]:
#Filter for dfirmID
flood_fset = flood_layer.query(where="CO_FIPS LIKE '063'")
flood_fset

In [ ]:
flood_fset.sdf.spatial.plot()

In [ ]:
#Find its spatial reference from the item
flood_sr = flood_item.spatialReference
flood_sr

In [ ]:
#Define an area to subset the data - we'll use Durham
durham_results = geocode("Durham City, NC")
durham_coords = durham_results[0]['location']
durham_coords['spatialReference'] = {'wkid':4326}
durham_point = Point(durham_coords)
durham_point_prj = durham_point.project_as(flood_sr)
durham_filter = filters.intersects(durham_point_prj.buffer(1000))
durham_filter

In [ ]:
#Apply the filter
durham_flood_fset = flood_layer.query(
    geometry_filter=durham_point
)
type(durham_flood_fset)

In [ ]:
print(durham_point)

In [ ]:
m = gis.map('Durham, NC')
m.draw(durham_point_prj)
m.draw(durham_point_prj.buffer(1))
m